# Библиотеки

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from time import sleep
import json
import datetime
import time

from bs4 import BeautifulSoup
import pandas as pd

# Функции

In [4]:

def format_day(date):
    day = '0' + str(date.day) if len(str(date.day)) == 1 else str(date.day)
    month = '0' + str(date.month) if len(str(date.month)) == 1 else str(date.month)
    year = str(date.year)
    return '-'.join([year, month, day])

def form_url(word, country, since, until):
    p1 = 'https://twitter.com/search?q='
    p2 =  word + '%20near%3A"' + country + '"%20since%3A' + since + '%20until%3A' + until + '&src=typd'
    return p1 + p2

def increment_day(date, i):
    return date + datetime.timedelta(days=i)


def get_tweets_and_frequences(start, end, word, country, tweet_selector, delay):
    
    days = (end - start).days + 1
    
    for day in range(days):
        d1 = format_day(increment_day(start, 0) )
        d2 = format_day(increment_day(start, 1) )
        url = form_url(word, country, d1, d2)
        print(url)
        print(d1)
        driver.get(url)
        sleep(delay)

        try:
            found_tweets = driver.find_elements_by_css_selector(tweet_selector)
            increment = 10

            while len(found_tweets) >= increment:
                print('scrolling down to load more tweets')
                driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                sleep(delay)
                found_tweets = driver.find_elements_by_css_selector(tweet_selector)
                increment += 10

            twee_all.append(len(found_tweets))

            print('{} tweets found, {} total'.format(len(found_tweets), len(ids) ) )

            for tweet in found_tweets:
                try:
                    page = tweet.get_attribute('innerHTML')
                    soup = BeautifulSoup(page, "lxml")
                    id = soup.findAll('p')[0].text
                    ids.append((country, word, id))
                except StaleElementReferenceException as e:
                    print('lost element reference', tweet)

        except NoSuchElementException:
            print('no tweets on this day')

        start = increment_day(start, 1)

    twee_n.append((country, word, sum(twee_all) ) )
    print('all done here')
    return(twee_all, twee_n, ids)



def words_in_fun_get_tweets_and_frequences(words, start, end, country, tweet_selector, delay):
    for word in words:
        result = get_tweets_and_frequences(start, end, word, country, tweet_selector, delay)
    return result


def create_rubric_data_frame(texts, columns):
    return pd.DataFrame(texts, 
                        columns=columns)

# Переменные

In [5]:
tweet_selector = 'li.js-stream-item'
delay = 1 

# ids - все твитты
# twee_all - твитты по одному слову по каждому дню
# twee_n - количесво твиттов в сумме для слова по всему периоду

ids = []
twee_all = []
twee_n = []

### Языки перевода

In [6]:
# 1 - английский
words1 = ['War','Conflict','Military%20conflict','Terrorism','Terrorist%20act','Enemy','External%20enemy',
         'External%20threat','Terrorist%20actions','Terrorist%20attack','Terrorist','Immigrants','Cold%20war',
         'External%20conflict','Threat'
        ]

# 2 - русский
words2 = ['Война','Конфликт','Военный%20конфликт','Терроризм','Террористический%20акт','Враг','Внешний%20враг',
         'Внешняя%20угроза','Террористические%20действия','Террористическая%20атака','Террорист','Иммигранты',
         'Холодная%20война','Внешние%20конфликты','Угроза'
        ]

# 3 - Испанский
words3 = ['Guerra','Conflicto','Conflicto%20militar','Terrorismo','Acto%20terrorista','Enemigo','Enemigo%20externo',
         'Amenaza%20externa','Actos%20terroristas','Ataque%20terrorista','Terrorista','Inmigrados','Guerra%20fría',
         'Conflictos%20externos','Amenaza'
        ]

# 4 - Французский
words4 = ['Guerre','Conflit','Le%20conflit%20militaire','Terrorisme','Attentat%20terroriste','Ennemi','Ennemi%20du%20dehors',
         'La%20menace%20extérieure','Activités%20terroristes','Attaque%20terroriste','Terroriste','Immigrants',
         'Guerre%20froide','Conflits%20externes','Menace'
        ]

# 5 - Китайский
words5 = ['战争','冲突','军事冲突','恐怖主义','恐怖主义行为','敌人','外部敌人','外部威胁','恐怖行动','恐怖袭击','恐怖分子',
         '移民局','冷战','外部冲突','威胁'
        ]

# 6 - Армянский
words6 = ['պատերազմ','հակամարտությունը','ռազմական%20հակամարտություն','ահաբեկչություն','ահաբեկչական%20գործողություն',
         'թշնամին','արտաքին%20թշնամի','արտաքին%20սպառնալիք','ահաբեկչական%20գործողություններ','ահաբեկչական%20հարձակում',
         'ահաբեկիչ','ներգաղթածները','սառը%20պատերազմ','արտաքին%20հակամարտություններ','սպառնալիք'
        ]

# 7 - Азербайджанский
words7 = ['müharibə','münaqişə','hərbi%20münaqişə','terrorizm','terror%20aktı','düşmən','xarici%20düşmən','xarici%20təhlükə',
         'terror%20hərəkətləri','terrorçu%20hücum','terrorçusu','immiqrantlar','soyuq%20müharibə','xarici%20münaqişələr',
         'təhlükə'
        ]

# 8 - Греческий
words8 = ['ο%20πόλεμος','σύγκρουση','στρατιωτική%20σύγκρουση','η%20τρομοκρατία','τρομοκρατική%20επίθεση','ο%20εχθρός',
          'εξωτερικός%20εχθρός','η%20εξωτερική%20απειλή','τρομοκρατικές%20ενέργειες','τρομοκρατική%20επίθεση',
          'τρομοκράτης','οι%20μετανάστες','ο%20ψυχρός%20πόλεμος','εξωτερικές%20συγκρούσεις','απειλή'
        ]

# 9 - Грузинский
words9 = ['ომი','კონფლიქტი','სამხედრო%20კონფლიქტი','ტერორიზმი','ტერორისტული%20აქტი','მტერი','გარე%20მტერი',
          'გარე%20საფრთხე','ტერორისტული%20აქცია','ტერორისტული%20თავდასხმა','ტერორისტული','ემიგრანტები',
          'ცივი%20ომი','გარე%20კონფლიქტი','საფრთხე'
        ]

# 10 -  Немецкий
words10 = ['Krieg','Konflikt','militärischer%20Konflikt','Terrorismus','Terrorakt','Feind','Äußerer%20Feind',
          'Externe%20Bedrohung','Terroristische%20Aktionen','Terroranschlag','Terrorist','Einwanderer',
          'kalter%20Krieg','Externe%20Konflikte','Bedrohung'
        ]

# 11 - Японский
words11 = ['戦争','競合','軍事衝突','テロリズム','テロ行為','敵','外部の敵','外部の脅威','テロ行為','テロ攻撃',
          'テロリスト','移民','冷戦','外部の競合','脅威'
        ]

# 12 - Нидерландский
words12 = ['Oorlog','Conflict','Militair%20conflict','Terrorisme','Terreurdaad','Vijand','Externe%20vijand',
          'Externe%20dreiging','Terroristische%20actie','Terreuraanslag','Terroristisch','Immigranten',
          'Koude%20Oorlog','Extern%20conflict','Bedreiging'
        ]

# 13 - Польский
words13 = ['Walka','Zatarg','Konflikt%20zbrojny','Terroryzm','Atak%20terrorystyczny','Wróg','Zewnętrzny%20wróg',
           'Zagrożenie%20zewnętrzne','Akty%20terroru','Atak%20terrorystyczny','Terrorysta','Imigrant',
           'Zimna%20wojna','Zewnętrzne%20konflikty','Zagrożenie'
        ]

# 14 - Румынский
words14 = ['Război','Conflictul','Conflictul%20militar','Terorism','Act%20terorist','Dușman','Inamic%20extern',
           'Amenințare%20externă','Acțiuni%20teroriste','Atac%20terorist','Terorist','Imigranți','Război%20rece',
           'Conflicte%20externe','Amenințare'
        ]

# 15 - Словенский
words15 = ['Vojna','Spor','Vojaški%20konflikt','Terorizem','Teroristično%20dejanje','Sovražnik','Zunanji%20sovražnik',
           'Zunanja%20nevarnost','Teroristično%20dejanje','Teroristični%20napad','Terorist','Priseljenci',
           'Hladna%20vojna','Zunanje%20navzkrižje','Grožnja'
        ]

# 16 - Корейский
words16 = ['대전','충돌','군사%20충돌','테러','테러%20행위','적','외장%20적','외부%20위협','테러리스트%20활동','테러%20공격',
           '테러','이민자','냉전','외부%20충돌','위협'
        ]

# 17 - Шведский
words17 = ['Krig','Konflikt','Militär%20konflikt','Terrorism','Terrorattack','Fiende','Yttre%20fiende','Yttre%20hot',
           'Terroristhandlingar','Terrorattack','Terrorist','Invandrare','Kalla%20kriget','Extern%20konflikt','Hot'
        ]

# 18 - Тайский
words18 = ['สงคราม','ความขัดแย้ง','ทหารข้อขัดแย้ง','การก่อการร้าย','ผู้ก่อการร้ายทำ','ศัตรู','เว็บเบราว์เซอร์ภายนอกศัตรู',
           'เว็บเบราว์เซอร์ภายนอกภัยคุกคาม','ผู้ก่อการร้ายทำ','ผู้ก่อการร้ายโจมตี','ผู้ก่อการร้าย','ผู้อพยพ','สงครามเย็น',
           'เว็บเบราว์เซอร์ภายนอกความขัดแย้ง','ภัยคุกคาม'
        ]

# 19 - Турецкий
words19 = ['Savaş','Çatışma','Askeri%20çatışma','Terörizm','Terör%20eylemi','Düşman','Dış%20düşman','Dış%20tehdit',
           'Terörist%20eylemler','Terörist%20saldırı','Terörist','Göçmenler','Soğuk%20Savaş','Dış%20çatışmalar',
           'Tehdit'
        ]

### Страны + периоды + список слов (на нужном языке)

In [7]:
# страна, начало года, конец года, список кодовых слов

countries = [
    1 - английский
    ['Australia', datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31), words1],
    ['Ghana', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words1],
    ['India', datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31), words1],
    ['Malaysia', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words1],
    ['New%20Zealand', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words1],
    ['Nigeria', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words1],
    ['Pakistan', datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31), words1],
    ['Philippines', datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31), words1],
    ['Rwanda', datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31), words1],
    ['Singapore', datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31), words1],
    ['South%20Africa', datetime.datetime(2013, 1, 1), datetime.datetime(2013, 12, 31), words1],
    ['Trinidad%20and%20Tobago', datetime.datetime(2010, 1, 1), datetime.datetime(2010, 12, 31), words1],
    ['United%20States', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words1], 
    ['Zimbabwe', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words1],
    
    # 1 - английский (вместо арабского)
    ['Algeria', datetime.datetime(2014, 1, 1), datetime.datetime(2014, 12, 31), words1],
    ['Egypt', datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31), words1],
    ['Iraq', datetime.datetime(2013, 1, 1), datetime.datetime(2013, 12, 31), words1],
    ['Jordan', datetime.datetime(2014, 1, 1), datetime.datetime(2014, 12, 31), words1],
    ['Kuwait', datetime.datetime(2013, 1, 1), datetime.datetime(2013, 12, 31), words1],
    ['Lebanon', datetime.datetime(2013, 1, 1), datetime.datetime(2013, 12, 31), words1],
    ['Libya', datetime.datetime(2013, 1, 1), datetime.datetime(2013, 12, 31), words1],
    ['Morocco', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words1],
    ['Palestine', datetime.datetime(2013, 1, 1), datetime.datetime(2013, 12, 31), words1],
    ['Qatar', datetime.datetime(2010, 1, 1), datetime.datetime(2010, 12, 31), words1],
    ['Tunisia', datetime.datetime(2013, 1, 1), datetime.datetime(2013, 12, 31), words1],
    ['Yemen', datetime.datetime(2013, 1, 1), datetime.datetime(2013, 12, 31), words1],
    
    # 2 - русский
    ['Belarus', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words2],
    ['Estonia', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words2],
    ['Kazakhstan', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words2],
    ['Kyrgyzstan', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words2],
    ['Russia', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words2],
    ['Ukraine', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words2],
    ['Uzbekistan', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words2],
    
    # 3 - испанский
    ['Argentina', datetime.datetime(2013, 1, 1), datetime.datetime(2013, 12, 31), words3],
    ['Brazil', datetime.datetime(2014, 1, 1), datetime.datetime(2014, 12, 31), words3],
    ['Colombia', datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31), words3],
    ['Chile', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words3],
    ['Ecuador', datetime.datetime(2013, 1, 1), datetime.datetime(2013, 12, 31), words3],
    ['Mexico', datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31), words3],
    ['Peru', datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31), words3],
    ['Spain', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words3],
    ['Uruguay', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words3],
    
    # 4 - французский
    ['Haiti', datetime.datetime(2016, 1, 1), datetime.datetime(2016, 12, 31), words4],
    
    # 5 - китайский
    ['China', datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31), words5],
    ['Hong%20Kong', datetime.datetime(2013, 1, 1), datetime.datetime(2013, 12, 31), words5],
    ['Taiwan', datetime.datetime(2013, 1, 1), datetime.datetime(2013, 12, 31), words5],
    
    # 6 - армянский
    ['Armenia', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words6],
    
    # 7 - азербайджанский
    ['Azerbaijan', datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31), words7],
    
    # 8 - греческий
    ['Cyprus', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words8],
    
    # 9 - грузинский
    ['Georgia', datetime.datetime(2014, 1, 1), datetime.datetime(2014, 12, 31), words9],
    
    # 10 - немецкий
    ['Germany', datetime.datetime(2013, 1, 1), datetime.datetime(2013, 12, 31), words10],
    
    # 11 - японский
    ['Japan', datetime.datetime(2010, 1, 1), datetime.datetime(2010, 12, 31), words11],
    
    # 12 - нидерландский
    ['Netherlands', datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31), words12],
    
    # 13 - польский
    ['Poland', datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31), words13],
    
    # 14 - румынский
    ['Romania', datetime.datetime(2012, 1, 1), datetime.datetime(2012, 12, 31), words14],
    
    # 15 - словенский
    ['Slovenia', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words15],
    
    # 16 - корейский
    ['South%20Korea', datetime.datetime(2010, 1, 1), datetime.datetime(2010, 12, 31), words16],
    
    # 17 - шведский
    ['Sweden', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words17],
    
    # 18 - тайский
    ['Thailand', datetime.datetime(2013, 1, 1), datetime.datetime(2013, 12, 31), words18],
    
    # 19 - турецкий
    ['Turkey', datetime.datetime(2011, 1, 1), datetime.datetime(2011, 12, 31), words19],
]

# Реализация

### Последовательность реализации:
* английсикий;
* английский (вместо арабского);
* русский;
* испанский;
* французский;
* китайский;
* армянский;
* азербайджанский;
* греческий;
* грузинский;
* немецкий;
* японский;
* нидерландский;
* польский;
* румынский;
* словенский;
* корейский;
* шведский;
* тайский;
* турецкий.

In [8]:
driver = webdriver.Chrome()

In [ ]:
for i in range(len(countries)):
    country = countries[i][0]
    start = countries[i][1]
    end = countries[i][2]
    words = countries[i][3]
    res = words_in_fun_get_tweets_and_frequences(words, start, end, country, tweet_selector, delay)

https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-01-01%20until%3A2014-01-02&src=typd
2014-01-01
3 tweets found, 0 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-01-02%20until%3A2014-01-03&src=typd
2014-01-02
2 tweets found, 3 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-01-03%20until%3A2014-01-04&src=typd
2014-01-03
0 tweets found, 5 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-01-04%20until%3A2014-01-05&src=typd
2014-01-04
1 tweets found, 5 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-01-05%20until%3A2014-01-06&src=typd
2014-01-05
0 tweets found, 6 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-01-06%20until%3A2014-01-07&src=typd
2014-01-06
1 tweets found, 6 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-01-07%20until%3A2014-01-08&src=typd
2014-01-07
1 tweets found, 7 total
https://twitter.com/search?q=War%20near%3

1 tweets found, 31 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-03-02%20until%3A2014-03-03&src=typd
2014-03-02
0 tweets found, 32 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-03-03%20until%3A2014-03-04&src=typd
2014-03-03
1 tweets found, 32 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-03-04%20until%3A2014-03-05&src=typd
2014-03-04
0 tweets found, 33 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-03-05%20until%3A2014-03-06&src=typd
2014-03-05
1 tweets found, 33 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-03-06%20until%3A2014-03-07&src=typd
2014-03-06
1 tweets found, 34 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-03-07%20until%3A2014-03-08&src=typd
2014-03-07
3 tweets found, 35 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-03-08%20until%3A2014-03-09&src=typd
2014-03-08
0 tweets found, 38 total
https://t

0 tweets found, 54 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-05-01%20until%3A2014-05-02&src=typd
2014-05-01
0 tweets found, 54 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-05-02%20until%3A2014-05-03&src=typd
2014-05-02
1 tweets found, 54 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-05-03%20until%3A2014-05-04&src=typd
2014-05-03
1 tweets found, 55 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-05-04%20until%3A2014-05-05&src=typd
2014-05-04
0 tweets found, 56 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-05-05%20until%3A2014-05-06&src=typd
2014-05-05
1 tweets found, 56 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-05-06%20until%3A2014-05-07&src=typd
2014-05-06
2 tweets found, 57 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-05-07%20until%3A2014-05-08&src=typd
2014-05-07
0 tweets found, 59 total
https://t

1 tweets found, 87 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-06-30%20until%3A2014-07-01&src=typd
2014-06-30
0 tweets found, 88 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-07-01%20until%3A2014-07-02&src=typd
2014-07-01
0 tweets found, 88 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-07-02%20until%3A2014-07-03&src=typd
2014-07-02
0 tweets found, 88 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-07-03%20until%3A2014-07-04&src=typd
2014-07-03
0 tweets found, 88 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-07-04%20until%3A2014-07-05&src=typd
2014-07-04
1 tweets found, 88 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-07-05%20until%3A2014-07-06&src=typd
2014-07-05
2 tweets found, 89 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-07-06%20until%3A2014-07-07&src=typd
2014-07-06
1 tweets found, 91 total
https://t

1 tweets found, 131 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-08-29%20until%3A2014-08-30&src=typd
2014-08-29
0 tweets found, 132 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-08-30%20until%3A2014-08-31&src=typd
2014-08-30
2 tweets found, 132 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-08-31%20until%3A2014-09-01&src=typd
2014-08-31
0 tweets found, 134 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-09-01%20until%3A2014-09-02&src=typd
2014-09-01
0 tweets found, 134 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-09-02%20until%3A2014-09-03&src=typd
2014-09-02
0 tweets found, 134 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-09-03%20until%3A2014-09-04&src=typd
2014-09-03
0 tweets found, 134 total
https://twitter.com/search?q=War%20near%3A"Jordan"%20since%3A2014-09-04%20until%3A2014-09-05&src=typd
2014-09-04
0 tweets found, 134 total
h

...

0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-03-01%20until%3A2013-03-02&src=typd
2013-03-01
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-03-02%20until%3A2013-03-03&src=typd
2013-03-02
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-03-03%20until%3A2013-03-04&src=typd
2013-03-03
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-03-04%20until%3A2013-03-05&src=typd
2013-03-04
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-03-05%20until%3A2013-03-06&src=typd
2013-03-05
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-03-06%20until%3A2013-03-07&src=typd
2013-03-06
0 tweets found, 1255 total
https://twitter.com/search?q=External

0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-04-23%20until%3A2013-04-24&src=typd
2013-04-23
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-04-24%20until%3A2013-04-25&src=typd
2013-04-24
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-04-25%20until%3A2013-04-26&src=typd
2013-04-25
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-04-26%20until%3A2013-04-27&src=typd
2013-04-26
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-04-27%20until%3A2013-04-28&src=typd
2013-04-27
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-04-28%20until%3A2013-04-29&src=typd
2013-04-28
0 tweets found, 1255 total
https://twitter.com/search?q=External

0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-06-15%20until%3A2013-06-16&src=typd
2013-06-15
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-06-16%20until%3A2013-06-17&src=typd
2013-06-16
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-06-17%20until%3A2013-06-18&src=typd
2013-06-17
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-06-18%20until%3A2013-06-19&src=typd
2013-06-18
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-06-19%20until%3A2013-06-20&src=typd
2013-06-19
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-06-20%20until%3A2013-06-21&src=typd
2013-06-20
0 tweets found, 1255 total
https://twitter.com/search?q=External

0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-08-07%20until%3A2013-08-08&src=typd
2013-08-07
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-08-08%20until%3A2013-08-09&src=typd
2013-08-08
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-08-09%20until%3A2013-08-10&src=typd
2013-08-09
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-08-10%20until%3A2013-08-11&src=typd
2013-08-10
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-08-11%20until%3A2013-08-12&src=typd
2013-08-11
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-08-12%20until%3A2013-08-13&src=typd
2013-08-12
0 tweets found, 1255 total
https://twitter.com/search?q=External

0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-09-29%20until%3A2013-09-30&src=typd
2013-09-29
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-09-30%20until%3A2013-10-01&src=typd
2013-09-30
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-10-01%20until%3A2013-10-02&src=typd
2013-10-01
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-10-02%20until%3A2013-10-03&src=typd
2013-10-02
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-10-03%20until%3A2013-10-04&src=typd
2013-10-03
0 tweets found, 1255 total
https://twitter.com/search?q=External%20conflict%20near%3A"Kuwait"%20since%3A2013-10-04%20until%3A2013-10-05&src=typd
2013-10-04
0 tweets found, 1255 total
https://twitter.com/search?q=External

In [ ]:
df_n = create_rubric_data_frame(res[1], ['country', 'word', 'frequency'])

In [ ]:
df_text = create_rubric_data_frame(res[2], ['country', 'word', 'text'])

In [ ]:
df_n['right_freq'] = [list(df_n['frequency'])[0]] + [list(df_n['frequency'])[i] - 
                                                     list(df_n['frequency'])[i-1] for i in range(1, 
                                                                                                 len(df_n['frequency']
                                                                                                    )
                                                                                                )
                                                    ]

In [ ]:
df_mean = df_n.groupby('country')['right_freq'].mean()

In [ ]:
df_mean.to_excel('df_mean_main5.xlsx')
df_text.to_excel('df_text_main5.xlsx')
df_n.to_excel('df_n_main5.xlsx')

# Конец реализации

In [ ]:
driver.close()